In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist',with_info=True,as_supervised=True)
mnist_dataset

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


{'test': <_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'train': <_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [3]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_dir='/root/tensorflow_datasets/mnist/incomplete.K6SCU7_3.0.1/',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""

In [4]:
mnist_train,mnist_test = mnist_dataset['train'],mnist_dataset['test']
validation_samples = 0.1*mnist_info.splits['train'].num_examples
validation_samples = tf.cast(validation_samples,tf.int64)
test_samples = mnist_info.splits['test'].num_examples
test_samples = tf.cast(test_samples,tf.int64)
test_samples

<tf.Tensor: shape=(), dtype=int64, numpy=10000>

In [5]:
def scale(image,label):
    image = tf.cast(image,tf.float32)
    image/=255.
    return image,label

In [6]:
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)
Buffer_size = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(Buffer_size)
validation_data = shuffled_train_and_validation_data.take(validation_samples)
train_data = shuffled_train_and_validation_data.skip(validation_samples)
Batch_size=100
train_data = train_data.batch(Batch_size)
validation_data = validation_data.batch(validation_samples)
test_data = test_data.batch(test_samples)
validation_inputs,validation_targets = next(iter(validation_data))

In [7]:
input_size = 784
output_size = 10
hidden_layer = 50
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer,activation='relu'),
    tf.keras.layers.Dense(hidden_layer,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [9]:
num_epochs = 10
model.fit(train_data,epochs=num_epochs,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/10
540/540 - 9s - 17ms/step - accuracy: 0.8850 - loss: 0.4059 - val_accuracy: 0.9392 - val_loss: 0.2064
Epoch 2/10
540/540 - 5s - 8ms/step - accuracy: 0.9468 - loss: 0.1834 - val_accuracy: 0.9582 - val_loss: 0.1450
Epoch 3/10
540/540 - 7s - 12ms/step - accuracy: 0.9604 - loss: 0.1364 - val_accuracy: 0.9652 - val_loss: 0.1158
Epoch 4/10
540/540 - 10s - 18ms/step - accuracy: 0.9681 - loss: 0.1101 - val_accuracy: 0.9695 - val_loss: 0.1024
Epoch 5/10
540/540 - 8s - 16ms/step - accuracy: 0.9725 - loss: 0.0933 - val_accuracy: 0.9758 - val_loss: 0.0850
Epoch 6/10
540/540 - 4s - 7ms/step - accuracy: 0.9755 - loss: 0.0797 - val_accuracy: 0.9740 - val_loss: 0.0866
Epoch 7/10
540/540 - 5s - 9ms/step - accuracy: 0.9788 - loss: 0.0716 - val_accuracy: 0.9760 - val_loss: 0.0744
Epoch 8/10
540/540 - 3s - 6ms/step - accuracy: 0.9814 - loss: 0.0614 - val_accuracy: 0.9798 - val_loss: 0.0706
Epoch 9/10
540/540 - 6s - 11ms/step - accuracy: 0.9836 - loss: 0.0554 - val_accuracy: 0.9822 - val_loss: 0.

In [10]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 812ms/step - accuracy: 0.9713 - loss: 0.0945


In [11]:
print('Test Loss:{0:2f}. Test accuracy:{1:2f}%'.format(test_loss,test_accuracy*100))

Test Loss:0.094508. Test accuracy:97.130001%
